In [1]:
import tensorflow as tf
import keras as K

Using TensorFlow backend.


In [2]:
print (tf.__version__, K.__version__)

1.12.0 2.2.4


In [3]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import cv2 as cv
%pylab inline

import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg

import tensorflow as tf
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU
# import keras_utils

Populating the interactive namespace from numpy and matplotlib


In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [5]:
BATCH_SIZE = 64
INIT_LR = 5e-3 

In [6]:
!cd ../PetImages/ ; ls

Cat  Dog


In [7]:
path = '../PetImages/'
imgs = []

In [8]:
for path, subdirs, files in os.walk(path):
    for name in files:
        imgs.append(os.path.join(path, name))

In [9]:
print (imgs[:5])

['../PetImages/Dog/11281.jpg', '../PetImages/Dog/976.jpg', '../PetImages/Dog/2678.jpg', '../PetImages/Dog/5831.jpg', '../PetImages/Dog/9492.jpg']


In [10]:
lbls = []
for img in imgs:
    if 'Cat' in img:
        lbls.append(0)
    else:
        lbls.append(1)

In [11]:
print (len(imgs), len(lbls))

24934 24934


In [12]:
lbls = keras.utils.to_categorical(lbls, num_classes=2)

In [13]:
(X_train, X_test, y_train, y_test) = train_test_split(imgs,
                                                      lbls,
                                                      test_size=0.2,
                                                      random_state=42)

In [14]:
for ind in range(0,5):
    print (X_train[ind], y_train[ind])

../PetImages/Dog/9700.jpg [0. 1.]
../PetImages/Cat/6336.jpg [1. 0.]
../PetImages/Dog/4734.jpg [0. 1.]
../PetImages/Cat/3070.jpg [1. 0.]
../PetImages/Dog/6812.jpg [0. 1.]


In [15]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])
# seq = iaa.Sequential([
#   iaa.Fliplr(0.5, name="Flipper"),
#   iaa.GaussianBlur((0, 3.0), name="GaussianBlur"),
#   iaa.Dropout(0.02, name="Dropout"),
#   iaa.AdditiveGaussianNoise(scale=0.01*255, name="MyLittleNoise"),
#   iaa.AdditiveGaussianNoise(loc=32, scale=0.0001*255, name="SomeOtherNoise"),
#   iaa.Affine(translate_px={"x": (-40, 40)}, name="Affine")
# ])

In [16]:
import numpy as np
from skimage.io import imread, imsave
from skimage.transform import resize

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(150,150), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def read_and_resize(self, filepath):
        img = imread(filepath)
#         img = preprocess_input(img)
        res = resize(img, self.dim, preserve_range=True, mode='reflect')
        return np.expand_dims(res, 0)
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = [self.read_and_resize(self.list_IDs[i])
             for i in indexes]
        y = self.labels[indexes]
        X = np.vstack(X)
        return seq.augment_images(X), y

<a>https://www.coursera.org/learn/convolutional-neural-networks/lecture/HAhz9/resnets</a><br>
<a>https://www.coursera.org/learn/convolutional-neural-networks/lecture/XAKNO/why-resnets-work</a>

In [17]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def resnet():
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    pred = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=pred)
    return model

In [18]:
# from keras.metrics import top_k_categorical_accuracy
# with tf.device("/cpu:0"):
#     model = Model(inputs=base_model.input, outputs=pred)
K.clear_session()
model = resnet()

/root/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [19]:
model.compile(optimizer = 'Adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs/resnet2', 
                                          histogram_freq=0, 
                                          batch_size=BATCH_SIZE, 
                                          write_graph=True, 
                                          write_grads=True, 
                                          write_images=True, 
                                          embeddings_freq=0,
                                          embeddings_layer_names=None, 
                                          embeddings_metadata=None) 

In [ ]:
# train_gen = DataGenerator(X_train, y_train, batch_size = 32)
# val_gen = DataGenerator(X_test, y_test, batch_size = 32)
checkpoint = ModelCheckpoint('checkpoint.h5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)
earlyStopping = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')

In [ ]:

params = {
            'dim': (224,224),
            'batch_size': 32,
            'n_classes': 2,
            'n_channels': 3,
            'shuffle': True
         }

train_gen = DataGenerator(X_train, y_train, **params)
val_gen = DataGenerator(X_test, y_test, **params)

In [ ]:
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback pour afficher la learning rate
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [ ]:
for layer in model.layers:
    layer.trainable = True
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.momentum = 0.8
    
for layer in model.layers[:-20]:
    layer.trainable = False

In [ ]:
history = model.fit_generator(generator = train_gen,
                          validation_data = val_gen,
                          use_multiprocessing = True,
#                           callbacks=[tensorboard, keras.callbacks.LearningRateScheduler(lr_scheduler),
#                                      LrHistory(), earlyStopping],
                          callbacks=[tensorboard],
                          workers=3,
                          epochs = 5)

/root/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/5


/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down

622/623 [============================>.] - ETA: 0s - loss: 0.5202 - acc: 0.7495

/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


...

Epoch 3/5
622/623 [============================>.] - ETA: 0s - loss: 0.2383 - acc: 0.9004

/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Epoch 3/5
623/623 [==============================] - 279s 447ms/step - loss: 0.2383 - acc: 0.9004 - val_loss: 7.7242 - val_acc: 0.5181


/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/root/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Epoch 4/5
216/623 [=========>....................] - ETA: 2:41 - loss: 0.2112 - acc: 0.9139

In [ ]:
# historique pour l'accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# historique pour la loss function
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# cols = 8
# rows = 3
# #pour allouer un cadre
# fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
# for i in range(cols):
#     for j in range(rows):
#         random_index = np.random.randint(0, len(y_train))
#         #on place chacune des images sur le cadre
#         ax = fig.add_subplot(rows, cols, i * rows + j + 1)
#         ax.grid('off')
#         ax.axis('off')
#         ax.imshow(x_train[random_index, :])
#         ax.set_title(cifar10_classes[y_train[random_index, 0]])
# plt.show()